In [1]:
#%%
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2
import pickle
import random
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F 

In [2]:
torch.cuda.is_available()

True

In [3]:
lr = 0.001
epochs = 500
batch_size = 5000

#IMPORTING DATA
#dataset variable should be a tuple of form (data, targets), each 1e5 long
with open('./datasets/list_orbital_dataset.pickle', 'rb') as data:
    dataset = pickle.load(data)

data, targets = dataset


#GET THE DEVICE (PREFEREABLY A GPU)
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

#CREATING A CUSTOM TRAINLOADER
class DatasetClass(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length

#CUSTOM TRAIN TEST SPLIT
def train_test_split_dataloaders(dataset, frac):
    train = torch.utils.data.Subset(dataset, range(0, int(frac*len(dataset))))
    test = torch.utils.data.Subset(dataset, range(int(frac*len(dataset)), len(dataset)))

    trainloader = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)
    testloader = DataLoader(test, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)

    return trainloader, testloader

dataset = DatasetClass(data, targets)
trainloader, testloader = train_test_split_dataloaders(dataset,0.3)

C:\Users\domin\AppData\Local\Temp/ipykernel_10944/4082650391.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  self.x = torch.tensor(x,dtype=torch.float32)


In [4]:
#DEFINE A NEURAL NETWORK
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(50,10)
        self.fc2 = nn.Linear(10,6)
        # self.fc3 = nn.Linear(10,1)

    def forward(self,x):
        x = x.view(batch_size, -1)
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        # x = F.sigmoid(self.fc3(x))
        return x

#A DIFFERENT ITERATION OF THE NETWORK
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # print('input shape', input_shape)
        self.cl1 = nn.Conv1d(1,3,3,1)

        self.cl2 = nn.Conv1d(3,5,3,1)
        self.fc1 = nn.Linear(,6)
    def forward(self,x):
        x = x.view(batch_size, 1)
        print('batch input into the model', x.size())
        x = torch.relu(self.cl1(x))
        x = torch.max_pool1d(x,3)
        x = torch.relu(self.cl2(x))
        print('after first conv1d', x.size())
        x = torch.max_pool1d(x,3)
        print('after second conv1d', x.size())
        x = x.view(batch_size, -1)
        print('size after flattening the conv layers', x.size())
        x = F.sigmoid(self.fc1(x))
        return x


#INSTANTIATE THE MODEL
model = NeuralNetwork()
print('model name: ', model)
device = get_device()
print('device being used: ', device)
model.to(device)

#OPTIMIZER AND LOSS FUNCTION
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
loss_fn = nn.MSELoss()
accuracy, train_losses, test_losses = [],[],[]

model name:  NeuralNetwork(
  (fc1): Linear(in_features=50, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=6, bias=True)
)
device being used:  cuda:0


In [96]:
#TRAINING LOOP
for i in range(epochs):
    for j, (x_train, y_train) in enumerate(trainloader):
        optimizer.zero_grad()
        output = model(x_train.to(device))

        #calculate loss
        loss = loss_fn(output, y_train.to(device))

        #backpropagation
        loss.backward()
        optimizer.step()

    if i%50==0:
        train_losses.append(loss)
        # accuracy.append(acc)
        print(f'epoch:{i}, loss:{loss}')        

print('Finished!')

TypeError: softmax() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype)
 * (Tensor input, name dim, *, torch.dtype dtype)


In [92]:

for j, (x_test, y_test) in enumerate(testloader):
    output = model(x_test.to(device))

    print(output, y_test)
    break


tensor([[0.1750, 1.0000, 0.9236, 1.0000, 0.2444, 1.0000],
        [0.1750, 1.0000, 0.9236, 1.0000, 0.2444, 1.0000],
        [0.1750, 1.0000, 0.9236, 1.0000, 0.2444, 1.0000],
        ...,
        [0.1750, 1.0000, 0.9236, 1.0000, 0.2444, 1.0000],
        [0.1750, 1.0000, 0.9236, 1.0000, 0.2444, 1.0000],
        [0.1750, 1.0000, 0.9236, 1.0000, 0.2444, 1.0000]], device='cuda:0',
       grad_fn=<SigmoidBackward0>) tensor([[0.3297, 0.7984, 0.9758, 0.6918, 0.3111, 4.9062],
        [0.0341, 0.4528, 1.1298, 0.0983, 0.1989, 4.3417],
        [0.1179, 3.0242, 1.4361, 1.2545, 0.3100, 4.1648],
        ...,
        [0.0553, 3.1321, 0.2169, 5.8220, 0.2745, 1.5065],
        [0.3205, 0.2641, 0.8986, 4.2232, 0.2045, 3.3947],
        [0.1678, 5.2859, 1.0647, 5.7189, 0.2894, 3.1746]])
